In [ ]:
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)
from pprint import PrettyPrinter
import time
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import seaborn as sns
from pymongo import MongoClient
from sklearn.metrics import mean_absolute_error
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.ar_model import AutoReg

In [ ]:
# Connecting to MongoDB and extracting the collection "dar-es-salaam"
client = MongoClient(host="localhost", port= 27017)
db = client["air-quality"]
dar = db["dar-es-salaam"]

In [ ]:
# Setting up PrettyPrinter, a tool to print the collections from MongoDB in order to get familiar with the db
pp= PrettyPrinter(indent=2)
result = dar.find_one({})
pp.pprint(result)

In [ ]:
# With this command we will know how many distinct sites there are
sites = dar.distinct("metadata.site")
sites

In [ ]:
# Aggregation to obtain the id of the sites and their counts 
result = dar.aggregate(
    [
        {"$group": {"_id": "$metadata.site", "count": {"$count": {}}}}
    ]
)
readings_per_site = list(result)
readings_per_site

In [ ]:
# Setting up a wrangle function to perform different transformations to the data in order to work with it
def wrangle(collection):
    results = collection.find(
        {"metadata.site": 11, "metadata.measurement": "P2"},
        projection={"P2": 1, "timestamp": 1, "_id": 0},
    )

    # Read results into DataFrame
    df = pd.DataFrame(list(results)).set_index("timestamp")

    # Localize timezone
    df.index = df.index.tz_localize("UTC").tz_convert("Africa/Dar_es_Salaam")

    # Remove outliers
    df = df[df["P2"] < 100]
    
    # Resample and forward-fill
    y = df["P2"].resample("1H").mean().fillna(method="ffill")
    
    return y

In [ ]:
# Let's all the registers from the dar collection, cleaned
y = wrangle(dar)
y.head()

In [ ]:
# Creating a time series plot with the data from y
fig, ax = plt.subplots(figsize=(15, 6))
y.plot(xlabel= "Date", ylabel="PM2.5 Level", title= "Dar es Salaam PM2.5 Levels", ax=ax);

In [ ]:
# Rolling average time series plot 
fig, ax = plt.subplots(figsize=(15, 6))
y.rolling(168).mean().plot(ax=ax, ylabel="PM2.5", title="Weekly Rolling Average");

In [ ]:
# ACF plot to check the autocorrelation of the lags
fig, ax = plt.subplots(figsize=(15, 6))
plot_acf(y, ax=ax)
plt.xlabel("Lag [hours]")
plt.ylabel("Correlation Coefficient");

In [ ]:
# Partial ACF to understand better the autocorrelation of the lags
fig, ax = plt.subplots(figsize=(15, 6))
plot_pacf(y, ax=ax)
plt.xlabel("Lag [hours]")
plt.ylabel("Correlation Coefficient");

In [ ]:
# Splitting the data in the train and test set
cutoff_test = int(len(y)* 0.9)
y_train = y.iloc[:cutoff_test]
y_test = y.iloc[cutoff_test:]
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

In [ ]:
# Creating a baseline with the average of the y_train data
y_train_mean = y_train.mean()
y_pred_baseline = [y_train_mean] * len(y_train)
mae_baseline = mean_absolute_error(y_train, y_pred_baseline)

print("Mean P2 Reading:", y_train_mean)
print("Baseline MAE:", mae_baseline)

In [ ]:
# The function to build and calculate our model with ARIMA 
# (as I will set the I and the MA to 0, it will be an ARMA model) to speed up the process, it will be an AR model
# Create range to test different lags
p_params = range(1, 31)

# Create empty list to hold mean absolute error scores
maes = []

# Iterate through all values of p in `p_params`
for p in p_params:
    
    order = (p, 0)
    
    # Build model
    model = AutoReg(y_train, lags=p).fit()
    
    # starting the time
    start_time = time.time()
    
    # time elapsed
    elapsed_time = round(time.time() - start_time, 2)
    print(f"Trained AR MODEL {order}, in {elapsed_time} seconds")

    # Make predictions on training data, dropping null values caused by lag
    y_pred = model.predict().dropna()

    # Calculate mean absolute error for training data vs predictions
    mae = mean_absolute_error(y_train.iloc[p:], y_pred)

    # Append `mae` to list `maes`
    maes.append(mae)

# Put list `maes` into Series with index `p_params`
mae_series = pd.Series(maes, name="mae", index=p_params)

# Inspect head of Series
mae_series.head()

In [ ]:
# Let's check the different errors of the models, to check which one performs better
print(mae_series)

In [ ]:
# There will be some tradeoff between the lowest MAE and the lower number of lags that we can have,
# so I decide that the one with 26 is the best performing model
best_p = 26
best_model = AutoReg(y_train, lags=best_p).fit()

In [ ]:
# Let´s check the residuals from the model
y_train_resid = model.resid
y_train_resid.name = "residuals"
y_train_resid.head()

In [ ]:
# Let's plot the histogram of the residuals
y_train_resid.hist()
plt.xlabel("Residuals")
plt.ylabel("Frequency")
plt.title("Best Model, Training Residuals");
# They follow more or less a normal distribution, so they are good

In [ ]:
# Now let's check the autocorrelation of the residuals
fig, ax = plt.subplots(figsize=(15, 6))
plot_acf(y_train_resid, ax=ax)
plt.xlabel("Lag [hours]")
plt.ylabel("Correlation Coefficient")
plt.title("Dar es Salaam, Training Residuals ACF");
# They are perfectly fine (no autocorreation)

In [ ]:
# Once we have calculated a good time series model, let's do some predictions using the command "forecast"
y_pred_wfv = pd.Series()
history = y_train.copy()
for i in range(len(y_test)):
    model = AutoReg(history, lags=best_p).fit()
    next_pred = model.forecast()
    y_pred_wfv = y_pred_wfv.append(next_pred)
    history = history.append(y_test[next_pred.index])
y_pred_wfv.name = "prediction"
y_pred_wfv.index.name = "timestamp"
y_pred_wfv.head()

In [ ]:
# At last, let's compare the original series with the prediction
df_pred_test = pd.DataFrame({"y_test": y_test, "y_pred_wfv": y_pred_wfv})
fig = px.line(df_pred_test, labels={"value": "PM2.5"})
fig.update_layout(
    title="Dar es Salaam, WFV Predictions",
    xaxis_title="Date",
    yaxis_title="PM2.5 Level",
)